In [1]:
import os
import sys
import numpy as np
import cv2

from utils.homography import compute_h, cor_p
from utils.triangulation import triangulation
from pathlib import Path
from shapely.geometry import Polygon

In [2]:
num_cams = 4
cam_poses = {} # key: cami, value: pose
for i in range(num_cams):
    with open(f'./camera_poses/{i:05d}.txt', 'r') as f:
        lines = f.readlines()
        pose = []
        for line in lines:
            data = list(map(float, line.split(" ")))
            pose.append(data)
        pose = np.array(pose)
        cam_poses[f'cam{i}'] = pose.reshape(4, 4)

In [3]:
yolo_outputs = {} # key: cami, value: yolo_output

for i in range(num_cams):

    # outputs of one camera:
    yolo_output = {} # key: class, value: yolo_output
    yolo_output['chair'], yolo_output['table'], yolo_output['person'] = [], [], []

    with open(f'./runs/detect/layout/cam{i}/labels/00000.txt', 'r') as f:
        lines = f.readlines()
        for line in lines:
            data = list(map(float, line.split(" ")))

            # check object class
            if data[0] == 56 or data[0] == 11: # chair or bench
                yolo_output['chair'].append(np.array(data[1:5]))
            elif data[0] == 60: # table
                yolo_output['table'].append(np.array(data[1:5]))
            elif data[0] == 0: # person
                yolo_output['person'].append(np.array(data[1:5]))
        yolo_outputs[f'cam{i}'] = yolo_output

In [7]:
h, w = 1456, 1928

poses = [cam_poses[f'cam{i}'][:3, :] for i in range(num_cams)]

K = np.array([
        [975.813843, 0, 960.973816],
        [0, 975.475220, 729.893921],
        [0, 0, 1]
    ])

for idx in range(len(cor_p)):
    X = [cor_p[i][idx][0] for i in range(num_cams)]
    Y = [cor_p[i][idx][1] for i in range(num_cams)]
    print(X, Y)
    triangulated = triangulation(poses, X, Y)
    print(triangulated)
    for i in range(num_cams):
        pose = poses[i]
        pose = np.vstack((pose, np.array([0, 0, 0, 1])))
        pose = np.linalg.inv(pose)
        pose = pose[:3, :]
        reversed = K @ pose @ np.array(triangulated.tolist() + [1]).reshape(4, 1)
        reversed = [reversed[0]/reversed[-1], reversed[1]/reversed[-1]]
        print(f"Reversed{i}: ", reversed)

[609, 1223, 1613, 1673] [1025, 1082, 893, 698]
[-0.64439422  0.64644906  1.9002724 ]
Reversed0:  [array([630.06925808]), array([1061.73846955])]
Reversed1:  [array([1253.68906216]), array([1108.11739441])]
Reversed2:  [array([1632.05232256]), array([904.23285742])]
Reversed3:  [array([1673.65826606]), array([698.05559345])]
[399, 568, 834, 1203] [632, 863, 897, 888]
[-2.03783136 -0.37931437  3.51591321]
Reversed0:  [array([395.38987451]), array([624.65475529])]
Reversed1:  [array([566.13197556]), array([862.1384292])]
Reversed2:  [array([835.80463896]), array([897.68442401])]
Reversed3:  [array([1204.04258058]), array([889.09437475])]
[1491, 1523, 1463, 1297] [819, 798, 669, 510]
[1.63704086 0.30526341 2.87154828]
Reversed0:  [array([1517.2754972]), array([833.59299404])]
Reversed1:  [array([1538.07191252]), array([805.5859874])]
Reversed2:  [array([1466.02533342]), array([670.76347106])]
Reversed3:  [array([1288.21413468]), array([506.31542233])]
[683, 718, 775, 834] [549, 739, 755, 7